In [ ]:
# --- Required Libraries ---
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.util import img_as_float
from skimage.metrics import structural_similarity as ssim_metric
from scipy.ndimage import gaussian_filter
from scipy.special import gamma

# --- PSNR ---
def psnr_paper(u, u0):
    mse = np.sum((u - u0)**2)
    dyn = (np.max(u0) - np.min(u0))**2
    return 10 * np.log10(dyn * u.size / (mse + 1e-8))

# --- Gradient Utilities ---
def compute_gradients(u):
    gradN = np.roll(u, -1, axis=0) - u
    gradS = np.roll(u, 1, axis=0) - u
    gradE = np.roll(u, -1, axis=1) - u
    gradW = np.roll(u, 1, axis=1) - u
    return gradN, gradS, gradE, gradW

def gradient_magnitude(u):
    gradN, _, gradE, _ = compute_gradients(u)
    return np.sqrt(gradN**2 + gradE**2)

# --- Diffusion Coefficient ---
def contrast_aware_diffusion(Gun, grad_Gun_mag, grad_ratio_mag, alpha, beta, lam, nu):
    M = np.max(Gun) or 1e-8
    contrast_term = np.clip(Gun / M, 1e-8, None) ** alpha
    denom = 1 + nu * (1 - lam) * (grad_Gun_mag ** beta) + nu * lam * (grad_ratio_mag ** beta)
    return contrast_term / denom

# --- Fractional-Time PDE (Ratio-Based, No DIP) ---
def ip_nde_fractional_no_reaction_dynamic_ratio(f, alpha, beta, lam, nu, sigma, T, tau, beta_c):
    epsilon = 1e-8
    u_hist = [f.copy(), f.copy()]
    u = f.copy()

    b = [1.0] + [((k + 1)**(1 - beta_c) - k**(1 - beta_c)) for k in range(1, T + 1)]
    b0 = b[0]
    g = gamma(2 - beta_c)

    ratio_at_1 = None
    ratio_mid = None

    for n in range(1, T + 1):
        u_prev = u_hist[-1]
        ratio_prior = np.clip(f / (u_prev + epsilon), 0.5, 2.0)
        if n == 1:
            ratio_at_1 = ratio_prior.copy()
        if n == T // 2:
            ratio_mid = ratio_prior.copy()

        Gun = gaussian_filter(u_prev, sigma=sigma)
        grad_mag_Gun = gradient_magnitude(Gun)
        grad_mag_ratio = gradient_magnitude(ratio_prior)

        g_map = contrast_aware_diffusion(Gun, grad_mag_Gun, grad_mag_ratio, alpha, beta, lam, nu)
        gradN_u, gradS_u, gradE_u, gradW_u = compute_gradients(u_prev)
        div = g_map * (gradN_u + gradS_u + gradE_u + gradW_u)

        frac_sum = np.zeros_like(u)
        for s in range(1, min(n, len(b))):
            u_n1s = u_hist[-s]
            u_ns = u_hist[-(s + 1)]
            frac_sum += b[s] * (u_n1s - u_ns)

        u_next = u_prev - (1 / b0) * (frac_sum - (tau**beta_c) * div * g)

        # Border Conditions
        u_next[0, :] = u_next[1, :]
        u_next[-1, :] = u_next[-2, :]
        u_next[:, 0] = u_next[:, 1]
        u_next[:, -1] = u_next[:, -2]

        u_hist.append(u_next)
        u = u_next

    final_ratio = np.clip(f / (u + epsilon), 0.5, 2.0)
    return np.clip(u, 0, 1), ratio_at_1, ratio_mid, final_ratio

# --- Speckle Noise ---
def add_gamma_speckle_noise(u, L=1):
    eta = np.random.gamma(shape=L, scale=1.0 / L, size=u.shape)
    return u * eta

# --- Main Execution ---
clean_img = img_as_float(imread(' your noisy image ')) #drop your noisy image here
if clean_img.ndim == 3:
    clean_img = clean_img.mean(axis=-1)

noisy_img = add_gamma_speckle_noise(clean_img, L=1)

PDE_PARAMS = dict(alpha=1.3, beta=1.5, lam=0.35, nu=1.0, sigma=0.9, T=100, tau=0.2, beta_c=0.90)
u_denoised, ratio_at_1, ratio_mid, ratio_final = ip_nde_fractional_no_reaction_dynamic_ratio(noisy_img, **PDE_PARAMS)

psnr_val = psnr_paper(u_denoised, clean_img)
ssim_val = ssim_metric(clean_img, u_denoised, data_range=1.0)
psnr_noisy = psnr_paper(noisy_img, clean_img)
ssim_noisy = ssim_metric(clean_img, noisy_img, data_range=1.0)

# --- Visualization ---
plt.figure(figsize=(30, 6))
plt.subplot(1, 6, 1); plt.imshow(clean_img, cmap='gray'); plt.title("Clean Image"); plt.axis('off')
plt.subplot(1, 6, 2); plt.imshow(noisy_img, cmap='gray'); plt.title(f"Noisy\nPSNR={psnr_noisy:.2f}, SSIM={ssim_noisy:.4f}"); plt.axis('off')
plt.subplot(1, 6, 3); plt.imshow(ratio_at_1, cmap='viridis'); plt.title("Ratio @ Iter 1"); plt.axis('off')
plt.subplot(1, 6, 4); plt.imshow(ratio_mid, cmap='viridis'); plt.title("Ratio @ Iter 50"); plt.axis('off')
plt.subplot(1, 6, 5); plt.imshow(ratio_final, cmap='viridis'); plt.title("Ratio @ Iter 100"); plt.axis('off')
plt.subplot(1, 6, 6); plt.imshow(u_denoised, cmap='gray'); plt.title(f"Denoised\nPSNR={psnr_val:.2f}, SSIM={ssim_val:.4f}"); plt.axis('off')
plt.tight_layout(); plt.show()
